# Ollama를 활용한 LLM prompt engineering 기초

- 작성자 임이로 bkksg.studio@gmail.com
- python 3.12.3사용중

In [1]:
!nvidia-smi #NVIDIA GPU 상태를 보여주는 표준 유틸리티. # MIG(Multi-Instance GPU) : Disabled
!python -c "import torch,platform,sys; print('py',sys.version); print('cuda?',torch.cuda.is_available())" #True면 PyTorch가 GPU(CUDA-그래픽카드를 ai연산에 사용하기 위함) 버전을 잘 찾았다는 뜻.

Tue Sep 16 13:39:14 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.86.15              Driver Version: 570.86.15      CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          Off |   00000000:C2:00.0 Off |                    0 |
| N/A   29C    P0             32W /  250W |       1MiB /  40960MiB |     N/A      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

#### ollama를 클라우드 서버에 설치

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh #https://ollama.com/ 

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#########                                                     29.2%
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


#### ollama serve 시작

In [3]:
import subprocess, os
# subprocess : 외부 명령어나 프로그램을 파이썬 코드에서 실행할 수 있게 해주는 모듈.
# os : 운영체제 관련 작업을 할 때 쓰는 모듈. / 환경변수 설정, 디렉토리 만들기/바꾸기, 경로 조작 등에 활용.

In [4]:
# # GPU 환경변수 설정 후 서버 실행
# !export OLLAMA_USE_GPU=1
# # 선택: 특정 GPU만 쓰려면 (예: 0번 GPU만)
# !export CUDA_VISIBLE_DEVICES=0

os.environ["OLLAMA_USE_GPU"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 필요 시 선택

In [16]:
# 주피터 프로세스 자체의 환경변수 수정
os.environ["OLLAMA_MODELS"] = "../../ollama" #절대경로 
# serve 띄울 때도 같은 os.environ 사용
subprocess.Popen(["ollama", "serve"], env=os.environ)


<Popen: returncode: None args: ['ollama', 'serve']>

time=2025-09-16T13:50:56.658+09:00 level=INFO source=routes.go:1332 msg="server config" env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:../../ollama OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICES:

In [17]:
!ollama list

[GIN] 2025/09/16 - 13:50:59 | 200 |     107.587µs |       127.0.0.1 | HEAD     "/"
[GIN] 2025/09/16 - 13:50:59 | 200 |    7.664782ms |       127.0.0.1 | GET      "/api/tags"
NAME           ID              SIZE     MODIFIED     
gemma3:27b     a418f5838eaf    17 GB    23 hours ago    
gpt-oss:20b    aa4295ac10c3    13 GB    23 hours ago    


#### ollama server 종료

In [14]:
subprocess.run(["pkill", "-f", "ollama serve"], check=False)

CompletedProcess(args=['pkill', '-f', 'ollama serve'], returncode=0)

#### ollama pull llm

In [19]:
# 모델 풀
subprocess.run(["ollama", "pull", "gpt-oss:20b"], check=True, env=os.environ)

[GIN] 2025/09/16 - 13:52:29 | 200 |      28.843µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/16 - 13:52:29 | 200 |  647.920254ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling b112e727c6f1: 100% ▕██████████████████▏  13 GB                         
pulling fa6710a93d78: 100% ▕██████████████████▏ 7.2 KB                         
pulling f60356777647: 100% ▕██████████████████▏  11 KB                         
pulling d8ba2f9a17b3: 100% ▕██████████████████▏   18 B                         
pulling 55c108d8e936: 100% ▕██████████████████▏  489 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gpt-oss:20b'], returncode=0)

In [20]:
# 모델 풀
subprocess.run(["ollama", "pull", "gemma3:27b"], check=True, env=os.environ)

[GIN] 2025/09/16 - 13:52:33 | 200 |      26.922µs |       127.0.0.1 | HEAD     "/"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ 

[GIN] 2025/09/16 - 13:52:33 | 200 |  695.745028ms |       127.0.0.1 | POST     "/api/pull"


pulling manifest 
pulling e796792eba26: 100% ▕██████████████████▏  17 GB                         
pulling e0a42594d802: 100% ▕██████████████████▏  358 B                         
pulling dd084c7d92a3: 100% ▕██████████████████▏ 8.4 KB                         
pulling 3116c5225075: 100% ▕██████████████████▏   77 B                         
pulling f838f048d368: 100% ▕██████████████████▏  490 B                         
verifying sha256 digest 
writing manifest 
success 


CompletedProcess(args=['ollama', 'pull', 'gemma3:27b'], returncode=0)

### ollama - gpt-oss:20b

In [21]:
!pip install -q langchain langchain-ollama
# 🔧 LangChain + Ollama 연동용 패키지 설치
# - langchain: 최신 LangChain 코어
# - langchain-ollama: Ollama 전용 커넥터
!pip install -q langchain-community
# 🔧 추가: community 모듈 설치 (체인/에이전트 등 확장 기능)

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1

In [22]:
import requests # HTTP 요청을 보내기 위한 라이브러리

# Ollama 서버에 텍스트 생성 요청 보내기
resp = requests.post(
    "http://127.0.0.1:11434/api/generate",  # 로컬에서 실행 중인 Ollama 서버 주소
    json={"model": "gpt-oss:20b", # 사용할 모델 이름
          "prompt": "장곡사 미륵불 괘불탱에 대해서 설명해줘.", # 모델에 보낼 프롬프트
          "stream": False},  # 스트리밍 모드 끄기 → 결과를 한 번에 받음
    timeout=120,  # 최대 120초까지 대기 (모델이 크면 응답이 느릴 수 있음)
)
# 모델이 생성한 응답 텍스트 출력
print(resp.json()["response"])

time=2025-09-16T13:54:55.332+09:00 level=INFO source=server.go:200 msg="model wants flash attention"
time=2025-09-16T13:54:55.332+09:00 level=INFO source=server.go:217 msg="enabling flash attention"
time=2025-09-16T13:54:55.333+09:00 level=INFO source=server.go:399 msg="starting runner" cmd="/usr/local/bin/ollama runner --ollama-engine --model ../../ollama/blobs/sha256-b112e727c6f18875636c56a779790a590d705aec9e1c0eb5a97d51fc2a778583 --port 42223"
time=2025-09-16T13:54:55.333+09:00 level=INFO source=server.go:672 msg="loading model" "model layers"=25 requested=-1
time=2025-09-16T13:54:55.359+09:00 level=INFO source=runner.go:1254 msg="starting ollama engine"
time=2025-09-16T13:54:55.360+09:00 level=INFO source=runner.go:1289 msg="Server listening on 127.0.0.1:42223"
time=2025-09-16T13:54:55.761+09:00 level=INFO source=server.go:678 msg="system memory" total="1007.1 GiB" free="903.2 GiB" free_swap="8.0 GiB"
time=2025-09-16T13:54:55.762+09:00 level=INFO source=server.go:686 msg="gpu memor

[GIN] 2025/09/16 - 13:55:33 | 500 | 39.768222322s |       127.0.0.1 | POST     "/api/generate"


KeyboardInterrupt: 

In [23]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

# Ollama 연결
# LangChain의 LLM 인터페이스를 사용해 Ollama를 쉽게 불러오는 방법.
# 내부적으로도 결국 requests로 Ollama API 호출하지만, LangChain이 포맷/파싱/스트리밍을 편하게 해줌.
llm = ChatOllama(
    base_url="http://localhost:11434",  # ollama serve가 켜져 있는 주소
    model="gpt-oss:20b",
    temperature=0.1,
    num_ctx=2048,
)

# from langchain_community.llms import Ollama
# from langchain_core.messages import SystemMessage, HumanMessage

# llm = Ollama(
#     base_url="http://127.0.0.1:11434",  # ollama serve 실행 중인 주소
#     model="gpt-oss:20b",
#     temperature=0.3,
#     num_ctx=2048,
# )

In [13]:
# 단일 메시지
resp = llm.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)


[GIN] 2025/09/15 - 15:22:55 | 200 | 42.936034303s |       127.0.0.1 | POST     "/api/chat"
## 장곡사(장곡사)와 미륵불 괘불탱(미륵불 괘불탱)  

### 1. 장곡사란?
- **위치**: 경기도 가평군 장곡면(남한과 북한 경계인 비무장지대(DMZ) 근처)  
- **역사**: 7세기(고려시대 이전) 삼국시대에 창건된 사찰로, 수세기 동안 여러 차례 파괴·재건을 거쳤습니다.  
- **의의**: 비무장지대 인근에 위치해 있어 ‘평화와 통일’의 상징으로 자주 언급됩니다. 사찰 내부에는 고대 석조 건축물과 불교 유물들이 보존돼 있어 문화재로서의 가치가 큽니다.

### 2. 미륵불 괘불탱이란?
- **정의**: ‘괘불탱(괘불탱)’은 ‘석탑’ 또는 ‘석불탑’이라는 뜻으로, 석조로 만든 작은 탑형 구조물입니다.  
- **미륵불**: 불교에서 ‘미륵보살’은 미래에 나타나 세상을 구원할 인물로, 미륵불은 미륵보살을 상징하는 불상입니다.  
- **미륵불 괘불탱**: 장곡사에 보존된 미륵불을 주제로 한 5층 석조 탑입니다. 8세기(고려시대 이전) 건축으로 추정되며, 한국의 대표적인 고대 석조 건축물 중 하나입니다.

### 3. 건축적 특징
| 구분 | 내용 |
|------|------|
| **시대** | 8세기(고려시대 이전) |
| **형태** | 5층 석조 탑 (각층이 점점 작아지는 형태) |
| **재료** | 화강암(석재) |
| **높이** | 약 3.5m(정확한 측정은 변동 가능) |
| **특징** | 각 층마다 작은 돌문이 새겨져 있으며, 상단에는 작은 돌탑(탑형) 모양이 있습니다. 미륵불을 상징하는 조각이 없지만, 미륵보살의 상징적 의미가 담겨 있습니다. |

### 4. 문화재 지정
- **문화재 번호**: 국보 제112호  
- **지정 이유**: 고대 석조 건축물로서 예술적·역사적 가치가 높으며, 장곡사의 대표 유물로서 한국 불교 문화의 중요한 부분을 형성합니다.

###

In [18]:
# 시스템 message / propmt (역할지정 / 명령어 지정) -> 이부분을 작성을 잘해야함. https://wikidocs.net/285650 (위키독스 진행)

# llm = ChatOllama(base_url="http://127.0.0.1:11434", model="gpt-oss:20b", temperature=0.3)
resp = llm.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 15:27:35 | 200 | 26.693870477s |       127.0.0.1 | POST     "/api/chat"
## 장곡사(장곡사)와 미륵불·괘불탱(괘불탱) 개요

| 항목 | 내용 |
|------|------|
| **위치** | 경기도 파주시 장곡면, 인천광역시와 경계에 인접한 한강(임진강) 인근 |
| **설립 시기** | 7세기(고려 이전) 설립, 이후 수차례 재건 |
| **문화재 등재** | 미륵불(국보 제123호), 괘불탱(국보 제124호) |
| **주요 사찰 건물** | 대웅전(미륵불이 안치된 본당), 괘불탱(석탑), 사찰 복합 건물(종각, 사찰원 등) |

---

### 1. 장곡사의 역사와 문화적 의미

- **고려시대의 미륵불 신앙**  
  장곡사는 미륵불(미륵보살)을 중심으로 한 사찰로, 미륵보살이 인류를 구원할 미래의 부처라는 신앙이 강하게 자리 잡았습니다. 12세기 이후 미륵불이 대웅전 안에 안치되면서 장곡사는 미륵보살 신앙의 중심지로 부상했습니다.

- **전쟁과 재건**  
  20세기 초 일제강점기와 한국전쟁 시기에 사찰이 파괴되거나 훼손되었으나, 1950년대 이후 복원사업을 통해 현재의 모습을 되찾았습니다. 특히 1970년대에 국보 등재를 통해 보존이 강화되었습니다.

- **현대의 역할**  
  장곡사는 미륵보살 신앙을 계승하고, 한국 전통 불교 예술과 건축을 연구·전시하는 문화센터 역할도 수행합니다. 매년 미륵보살 축제(미륵보살제)가 열려 많은 신자와 관광객이 방문합니다.

---

### 2. 미륵불(국보 제123호)

| 항목 | 내용 |
|------|------|
| **시대** | 고려 12세기(약 1200년경) |
| **재료** | 대리석(또는 화강암) |
| **크기** | 높이 약 1.5m, 무게 약 3톤 |
| **특징** | 미륵보살이 손에 팔을 들고, 두 손에 팔을 올린 자세(팔을 올린 자세)로 서 있습니다. 미륵보살의 얼굴은 부드러운 미소와 

### ollama - gemma3:27b

In [24]:
# Ollama 연결
llm1 = ChatOllama(
    base_url="http://127.0.0.1:11434",  # ollama serve가 켜져 있는 주소
    model="gemma3:27b",
    temperature=0.1,
    num_ctx=2048,
)

In [20]:
# 단일 메시지
resp = llm1.invoke("장곡사 미륵불 괘불탱에 대해서 설명해줘.")
print(resp.content)

[GIN] 2025/09/15 - 15:28:13 | 200 | 20.615460119s |       127.0.0.1 | POST     "/api/chat"
장곡사 미륵불 괘불탱은 경상남도 거창군 북상면 장곡사에 있는 괘불탱으로, 조선 후기에 제작된 불교 회화의 걸작입니다. 괘불탱은 원래 절에서 중요한 의식을 치를 때 내걸어 놓는 그림으로, 장곡사 괘불탱은 특히 뛰어난 예술성과 역사적 가치를 인정받아 국보 제135호로 지정되었습니다.

**주요 특징:**

*   **미륵보살의 광대한 세계:** 괘불탱의 중심에는 미래에 인간 세계를 구제할 미륵보살이 웅장하게 자리 잡고 있습니다. 미륵보살은 화려한 보관을 쓰고, 온화한 미소를 짓고 있으며, 손에는 각각 지혜와 자비를 상징하는 물건을 들고 있습니다. 미륵보살 주변에는 다양한 보살과 천인, 그리고 극락 세계를 상징하는 연꽃과 구름 등이 섬세하게 묘사되어 있습니다.
*   **화려하고 섬세한 채색:** 괘불탱은 밝고 화려한 오방색을 사용하여 미륵보살의 장엄함과 극락 세계의 아름다움을 표현합니다. 특히, 옷의 주름이나 보살의 표정 등은 매우 섬세하게 묘사되어 있어, 당시 불화 장인의 뛰어난 기량을 엿볼 수 있습니다.
*   **조선 후기 불화의 특징:** 장곡사 괘불탱은 조선 후기 불화의 특징을 잘 보여줍니다. 즉, 사실적인 묘사보다는 이상적인 아름다움을 추구하고, 화려한 색채와 섬세한 문양을 사용하여 불교 세계의 신비로움을 표현합니다.
*   **제작 배경:** 장곡사 괘불탱은 18세기 후반에 제작된 것으로 추정됩니다. 당시 장곡사는 조선 영남 지역의 중요한 불교 중심지였으며, 괘불탱은 절의 권위를 상징하고 불교 의식을 거행하는 데 중요한 역할을 했습니다.

**장곡사 괘불탱 감상 포인트:**

*   **미륵보살의 표정과 자세:** 미륵보살의 온화한 미소와 자비로운 눈빛, 그리고 편안한 자세를 통해 보살의 자애로운 마음을 느껴보세요.
*   **보살과 천인의 다양한 모습:** 미륵보살 주변에 묘사된 보살과 천

In [21]:
resp = llm1.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괘불탱에 대해서 설명해줘."),
])
print(resp.content)

[GIN] 2025/09/15 - 15:28:51 | 200 | 26.613180367s |       127.0.0.1 | POST     "/api/chat"
안녕하세요! AI 문화유산 스마트 도슨트입니다. 장곡사 미륵불 괘불탱에 대해 자세히 설명해 드리겠습니다.

**장곡사 미륵불 괘불탱 (長谷寺 彌勒佛 掛佛탱)**

*   **소장처:** 전라북도 정읍시 장곡사
*   **지정:** 보물 제1358호 (2003년)
*   **제작시기:** 조선 후기 (추정)
*   **크기:** 가로 6.8m, 세로 10.3m
*   **재료:** 비단

**괘불탱이란?**

괘불탱은 불교 의식 때 법당에 걸어놓는 대형 불화입니다. 넓은 비단에 그려져 있으며, 주로 석가모니불의 영산회상(靈山會上) 장면을 묘사합니다. 영산회상은 석가모니 부처가 깨달음을 얻은 영산에서 설법하는 모습을 그린 것으로, 많은 보살과 천신, 나한 등이 함께 등장하는 장엄한 장면입니다.

**장곡사 미륵불 괘불탱의 특징**

장곡사 미륵불 괘불탱은 다른 괘불탱들과 몇 가지 독특한 특징을 가지고 있습니다.

*   **미륵불 중심:** 대부분의 괘불탱이 석가모니불을 중심으로 그려지는 반면, 이 괘불탱은 미륵불을 중심으로 조성되었다는 점이 특징적입니다. 미륵불은 미래에 내려와 중생을 구제할 부처로 여겨집니다.
*   **화려하고 섬세한 표현:** 괘불탱 전체적으로 화려하고 섬세한 표현이 돋보입니다. 특히, 미륵불의 얼굴과 몸, 옷의 주름, 보살들의 표정과 몸짓 등이 정교하게 묘사되어 있습니다.
*   **다양한 등장인물:** 미륵불을 중심으로 다양한 보살과 천신, 나한 등이 등장합니다. 이들은 각기 다른 자세와 표정을 통해 불교 세계의 다채로운 모습을 보여줍니다.
*   **권위적이고 장엄한 분위기:** 괘불탱 전체적으로 권위적이고 장엄한 분위기가 느껴집니다. 이는 불교 의식에서 괘불탱이 갖는 신성한 의미를 반영합니다.

**장곡사 괘불탱이 중요한 이유**

장곡사 미륵불 괘불탱은 조선 후기 불화의 